# Evaluación Comparativa de Backbones CNN para Clasificación Binaria

En este cuaderno se evalúan de forma automática varios modelos preentrenados (backbones) sobre nuestro conjunto de datos binario. El objetivo es determinar cuál de ellos ofrece el mejor rendimiento en términos de precisión, pérdida de validación, F1-Score y tiempo de entrenamiento, para seleccionarlo como base del entrenamiento final.

Este proceso es equivalente a un "LazyPredict" adaptado a Deep Learning con Keras.



## Paso 1 – Importación de librerías

Importamos todas las dependencias necesarias para definir los modelos, gestionar los datos y registrar los resultados.


In [1]:
import time
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    MobileNetV2, EfficientNetB0, ResNet50, DenseNet121,
    InceptionV3, NASNetMobile, Xception, VGG16
)


## Paso 2 – Rutas y carga de datos

Cargamos los conjuntos de entrenamiento, validación y prueba desde archivos CSV, y los preparamos para su uso mediante `ImageDataGenerator`. Esta fase incluye la normalización y el redimensionamiento de las imágenes.


In [9]:
from pathlib import Path
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Rutas de los CSVs
DATA_SPLIT_DIR = Path("../data/binario_split")
TRAIN_CSV = DATA_SPLIT_DIR / "train.csv"
VAL_CSV   = DATA_SPLIT_DIR / "val.csv"

# Parámetros
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Carga de los CSVs sin codificación
df_train = pd.read_csv(TRAIN_CSV)
df_val = pd.read_csv(VAL_CSV)

# Preprocesamiento de imágenes
datagen = ImageDataGenerator(rescale=1./255)

# Generador para entrenamiento
train_gen = datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col="filepath",      # columna con la ruta a la imagen
    y_col="label",         # etiqueta binaria como string
    directory=None,        # rutas ya incluidas en 'filepath'
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# Generador para validación
val_gen = datagen.flow_from_dataframe(
    dataframe=df_val,
    x_col="filepath",
    y_col="label",
    directory=None,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)



Found 6907 validated image filenames belonging to 2 classes.
Found 987 validated image filenames belonging to 2 classes.


## Paso 3 – Función constructora de modelos

Definimos una función que crea un modelo CNN con un backbone preentrenado y una capa densa de salida para clasificación binaria. Se congelan los pesos del modelo base para reducir el tiempo de entrenamiento.


In [10]:
def build_model(backbone_fn, input_shape=(224, 224, 3)):
    base = backbone_fn(include_top=False, weights="imagenet", input_shape=input_shape)
    base.trainable = False  # No entrenamos el backbone

    x = GlobalAveragePooling2D()(base.output)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base.input, outputs=output)
    model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model


## Paso 4 – Selección de modelos a comparar

Especificamos los modelos preentrenados que evaluaremos como posibles candidatos a utilizar en el entrenamiento final.


In [11]:
backbones = {
    "MobileNetV2": MobileNetV2,
    "EfficientNetB0": EfficientNetB0,
    "ResNet50": ResNet50,
    "DenseNet121": DenseNet121,
    "InceptionV3": InceptionV3,
    "NASNetMobile": NASNetMobile,
    "Xception": Xception,
    "VGG16": VGG16
}


## Paso 5 – Entrenamiento y evaluación de cada backbone

Entrenamos cada uno de los modelos por unas pocas épocas para tener una idea comparativa. Se mide el tiempo de entrenamiento, la precisión de validación, la pérdida y el F1-Score.


In [12]:
resultados = []

for nombre, backbone_fn in backbones.items():
    print(f"\n🔍 Entrenando backbone: {nombre}")
    start = time.time()

    model = build_model(backbone_fn, input_shape=(224, 224, 3))
    es = EarlyStopping(monitor="val_accuracy", patience=2, restore_best_weights=True)

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=5,
        callbacks=[es],
        verbose=0
    )

    val_acc = history.history["val_accuracy"][-1]
    val_loss = history.history["val_loss"][-1]

    # Evaluar F1
    val_gen.reset()
    y_true = val_gen.classes
    y_pred_prob = model.predict(val_gen)
    y_pred = (y_pred_prob > 0.5).astype(int)
    f1 = f1_score(y_true, y_pred)

    duracion = time.time() - start
    resultados.append({
        "Backbone": nombre,
        "Val Accuracy": round(val_acc, 4),
        "Val Loss": round(val_loss, 4),
        "F1 Score": round(f1, 4),
        "Tiempo (s)": int(duracion)
    })



🔍 Entrenando backbone: MobileNetV2


c:\Users\isard\ProyectosLocales\env_multiclase_fatiga\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 169ms/step

🔍 Entrenando backbone: EfficientNetB0
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 343ms/step

🔍 Entrenando backbone: ResNet50
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 698ms/step

🔍 Entrenando backbone: DenseNet121
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 30s 888ms/step

🔍 Entrenando backbone: InceptionV3
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 441ms/step

🔍 Entrenando backbone: NASNetMobile
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 360ms/step

🔍 Entrenando backbone: Xception
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 31s 972ms/step

🔍 Entrenando backbone: VGG16
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step


## Paso 6 – Comparativa final de resultados

Mostramos los resultados ordenados por F1 Score para facilitar la selección del mejor modelo.


In [13]:
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by="F1 Score", ascending=False).reset_index(drop=True)
df_resultados


,Backbone,Val Accuracy,Val Loss,F1 Score,Tiempo (s)
0,DenseNet121,0.6190,0.6517,0.5068,946
1,InceptionV3,0.6413,0.6361,0.4670,523
2,MobileNetV2,0.6353,0.6426,0.4395,187
3,NASNetMobile,0.6261,0.6526,0.4390,390
4,Xception,0.6596,0.6317,0.4233,1113
5,ResNet50,0.5988,0.6810,0.3956,852
6,VGG16,0.6272,0.6713,0.3917,2198
7,EfficientNetB0,0.5066,0.6942,0.0000,242


## Paso 7 – Guardado de resultados

Guardamos la tabla comparativa para documentación futura y trazabilidad del proceso de selección.


In [14]:
df_resultados.to_csv("../models/resumen_backbones.csv", index=False)
